# Caesar Cipher Cryptanalysis

Given ciphertext:

```
PRCSOFQX FP QDR AFOPQ CZSPR LA JFPALOQSKR. QDFP FP ZK LIU BROJZK MOLTROE.
```

Tasks:
1. Count the frequency of letters. List the top three most frequent characters.
2. Knowing that this is English, what are commonly used three-letter and two-letter words? Does this help in cracking the text?
3. Crack the given text. Measure the time taken to crack this message.
4. Create a simple Python program for cracking the Caesar cipher text using brute force. Explain the design and demonstrate the software.

In [2]:
from collections import Counter

ciphertext = "PRCSOFQX FP QDR AFOPQ CZSPR LA JFPALOQSKR. QDFP FP ZK LIU BROJZK MOLTROE."

# Count frequency of each letter (ignore non-letters)
letters = [c for c in ciphertext if c.isalpha()]
frequency = Counter(letters)

# Display frequencies and top 3
print("Letter frequencies:")
for char, count in frequency.most_common():
    print(f"{char}: {count}")

print("\nTop 3 most frequent characters:")
for char, count in frequency.most_common(3):
    print(f"{char}: {count}")

Letter frequencies:
P: 7
R: 6
O: 6
F: 6
Q: 5
L: 4
S: 3
A: 3
Z: 3
K: 3
C: 2
D: 2
J: 2
X: 1
I: 1
U: 1
B: 1
M: 1
T: 1
E: 1

Top 3 most frequent characters:
P: 7
R: 6
O: 6


## Common English Words and Cryptanalysis

Common two-letter words: `of`, `to`, `in`, `it`, `is`, `be`, `as`, `at`, `so`, `we`, `he`, `by`, `or`, `on`, `do`, `if`, `me`, `my`, `up`, `an`, `go`, `no`, `us`, `am`

Common three-letter words: `the`, `and`, `for`, `are`, `but`, `not`, `you`, `all`, `any`, `can`, `had`, `her`, `was`, `one`, `our`, `out`, `day`, `get`, `has`, `him`, `his`, `how`, `man`, `new`, `now`, `old`, `see`, `two`, `way`, `who`, `boy`, `did`, `its`, `let`, `put`, `say`, `she`, `too`, `use`

When brute-forcing a Caesar cipher, look for these words in the decrypted outputs. Their presence is a strong indicator of the correct shift.

In [3]:
import time

def caesar_decrypt(text, shift):
    result = ''
    for char in text:
        if char.isupper():
            result += chr((ord(char) - 65 - shift) % 26 + 65)
        elif char.islower():
            result += chr((ord(char) - 97 - shift) % 26 + 97)
        else:
            result += char
    return result

start = time.time()
print("Trying all possible Caesar cipher shifts:")
for shift in range(26):
    candidate = caesar_decrypt(ciphertext, shift)
    print(f"Shift {shift:2}: {candidate}")
end = time.time()
print(f"\nTime taken: {end - start:.4f} seconds")

Trying all possible Caesar cipher shifts:
Shift  0: PRCSOFQX FP QDR AFOPQ CZSPR LA JFPALOQSKR. QDFP FP ZK LIU BROJZK MOLTROE.
Shift  1: OQBRNEPW EO PCQ ZENOP BYROQ KZ IEOZKNPRJQ. PCEO EO YJ KHT AQNIYJ LNKSQND.
Shift  2: NPAQMDOV DN OBP YDMNO AXQNP JY HDNYJMOQIP. OBDN DN XI JGS ZPMHXI KMJRPMC.
Shift  3: MOZPLCNU CM NAO XCLMN ZWPMO IX GCMXILNPHO. NACM CM WH IFR YOLGWH JLIQOLB.
Shift  4: LNYOKBMT BL MZN WBKLM YVOLN HW FBLWHKMOGN. MZBL BL VG HEQ XNKFVG IKHPNKA.
Shift  5: KMXNJALS AK LYM VAJKL XUNKM GV EAKVGJLNFM. LYAK AK UF GDP WMJEUF HJGOMJZ.
Shift  6: JLWMIZKR ZJ KXL UZIJK WTMJL FU DZJUFIKMEL. KXZJ ZJ TE FCO VLIDTE GIFNLIY.
Shift  7: IKVLHYJQ YI JWK TYHIJ VSLIK ET CYITEHJLDK. JWYI YI SD EBN UKHCSD FHEMKHX.
Shift  8: HJUKGXIP XH IVJ SXGHI URKHJ DS BXHSDGIKCJ. IVXH XH RC DAM TJGBRC EGDLJGW.
Shift  9: GITJFWHO WG HUI RWFGH TQJGI CR AWGRCFHJBI. HUWG WG QB CZL SIFAQB DFCKIFV.
Shift 10: FHSIEVGN VF GTH QVEFG SPIFH BQ ZVFQBEGIAH. GTVF VF PA BYK RHEZPA CEBJHEU.
Shift 11: EGRHDUFM UE FSG PUDEF RO

## Design Explanation

- The program first counts letter frequencies to help with analysis.
- It lists common English two-letter and three-letter words, which are useful for identifying likely plaintext in brute-force results.
- The Caesar cipher is brute-forced by trying all 26 possible shifts and printing each result.
- You can visually inspect the outputs for recognizable English words, especially the common ones listed above.
- For more automation, you could use an English dictionary to check for valid words in each output and highlight the most likely plaintext.

In [ ]:
import urllib.request

try:
    with open('words_alpha.txt', 'r') as f:
        english_words = set(word.strip() for word in f)
except FileNotFoundError:
    url = 'https://raw.githubusercontent.com/dwyl/english-words/master/words_alpha.txt'
    urllib.request.urlretrieve(url, 'words_alpha.txt')
    with open('words_alpha.txt', 'r') as f:
        english_words = set(word.strip() for word in f)

def max_similarity(word, word_set):
    word = word.lower()
    if word in word_set:
        return 1.0
    max_sim = 0.0
    for w in word_set:
        if len(w) != len(word):
            continue
        sim = sum(a == b for a, b in zip(word, w)) / len(word)
        if sim > max_sim:
            max_sim = sim
    return max_sim

def check_words_similarity(text, word_set):
    words = [w.strip('.,') for w in text.split()]
    result = []
    for w in words:
        sim = max_similarity(w, word_set)
        result.append((w, sim))
    return result

sim_score = []
for shift in range(26):
    candidate = caesar_decrypt(ciphertext, shift)
    checked = check_words_similarity(candidate, english_words)
    total_sim = sum(sim * len(word) for word, sim in checked)
    total_chars = sum(len(word) for word, sim in checked)
    score = total_sim / total_chars if total_chars > 0 else 0
    # Print each word with its similarity
    print(f"Shift {shift:2} - {candidate} similarity score: {score:.2f}")
    sim_score.append((score, shift, candidate))

sim_score.sort(reverse=True)
print(f"Candidate with most score: {sim_score[0][2]} (Shift {sim_score[0][1]}, Score {sim_score[0][0]:.2f})")


Shift  0 - PRCSOFQX FP QDR AFOPQ CZSPR LA JFPALOQSKR. QDFP FP ZK LIU BROJZK MOLTROE. similarity score: 0.61
Shift  1 - OQBRNEPW EO PCQ ZENOP BYROQ KZ IEOZKNPRJQ. PCEO EO YJ KHT AQNIYJ LNKSQND. similarity score: 0.58
Shift  2 - NPAQMDOV DN OBP YDMNO AXQNP JY HDNYJMOQIP. OBDN DN XI JGS ZPMHXI KMJRPMC. similarity score: 0.56
Shift  3 - MOZPLCNU CM NAO XCLMN ZWPMO IX GCMXILNPHO. NACM CM WH IFR YOLGWH JLIQOLB. similarity score: 0.58
Shift  4 - LNYOKBMT BL MZN WBKLM YVOLN HW FBLWHKMOGN. MZBL BL VG HEQ XNKFVG IKHPNKA. similarity score: 0.58
Shift  5 - KMXNJALS AK LYM VAJKL XUNKM GV EAKVGJLNFM. LYAK AK UF GDP WMJEUF HJGOMJZ. similarity score: 0.59
Shift  6 - JLWMIZKR ZJ KXL UZIJK WTMJL FU DZJUFIKMEL. KXZJ ZJ TE FCO VLIDTE GIFNLIY. similarity score: 0.58
Shift  7 - IKVLHYJQ YI JWK TYHIJ VSLIK ET CYITEHJLDK. JWYI YI SD EBN UKHCSD FHEMKHX. similarity score: 0.59
Shift  8 - HJUKGXIP XH IVJ SXGHI URKHJ DS BXHSDGIKCJ. IVXH XH RC DAM TJGBRC EGDLJGW. similarity score: 0.54
Shift  9 - GITJFWHO WG HUI R